In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
import matplotlib.pylab as plt
import numpy as np
from torchvision import models
torch.manual_seed(2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available')
else:
    print('CUDA is available!')
    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Mount Google Drive

In [ ]:

trainpath = "/content/drive/My Drive/dataset/"
validationpath = "/content/drive/My Drive/datasetT/"


# Transfer train and validation data to 128x128 and then to tensor

In [ ]:
#Test Transform and data loader
transform1 = transforms.Compose([transforms.Resize((224,224)),transforms.CenterCrop(size=224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
train_dataset = dsets.ImageFolder(root=trainpath, transform=transform1)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=32, shuffle=True)

#validation transform and dataloader
validation_transform = transforms.Compose([transforms.Resize((224,224)),transforms.CenterCrop(size=224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
test_dataset = dsets.ImageFolder(root=validationpath, transform=validation_transform)
validation_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=32, shuffle=False)

num_epochs = 2
num_classes = 3
batch_size = 32
learning_rate = 0.001
print('there are {}  images in training set.'.format(len(train_dataset)))
print('there are {}  images in test set.'.format(len(test_dataset)))
print('there are {}  images in training set.'.format(len(train_loader)))
print('there are {}  images in training loader.'.format(len(validation_loader)))



# Load pretrained Model, unfreezing their fully connected layer

In [ ]:
model=models.googlenet(pretrained=True)
for param in model.parameters():
  for i in param:
    param.requires_grad = False
    num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

print(model.fc)

In [ ]:
from torchsummary import summary

summary(model.cuda(), (3, 224, 224))


# Using SGD for finding Optima

In [ ]:
model = model.to (device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss().to (device)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


# Traning the Model

In [ ]:
# Train the model
print(len(train_loader))
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        images=images.to (device)
        labels=labels.to (device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        print("Training")
    #if (i + 1) % 100 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}%'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),(correct / total) * 100))
PATH = '/content/drive/My Drive/googlenet_weights1/.pth'
torch.save(model.state_dict(), PATH)       


# Loading weights, if already trained 

In [ ]:
#PATH = '/content/drive/My Drive/--.pth'
#model.load_state_dict(torch.load(PATH))


# Finding accuracy

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in validation_loader:
        images=images.to (device)
        labels=labels.to (device)
        outputs = model(images).to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the  test images: {} %'.format((correct / total) * 100))
    

# Ploting Confusion matrix

In [ ]:
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(validation_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
sns.heatmap(conf_mat, annot=True,fmt="d",cmap='YlGnBu')

ax= plt.subplot()
# Per-class accuracy
#class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
#print(class_accuracy)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in validation_loader:
        images=images.to (device)
        labels=labels.to (device)
        outputs = model(images).to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the  test images: {} %'.format((correct / total) * 100))
    #sns.heatmap(cm, cmap="YlGnBu",annot=True, ax = ax,cbar=False); #annot=True to annotate cells


ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 